In [1]:
## install libraries
#!pip install geemap
#!pip install ipyleaflet


In [2]:
# import libraries
import ee
import ee; ee.Initialize()
import geemap
import os
import ipyleaflet
import numpy as np

In [3]:
#Define to area of study and dates
State = 'Minnesota'
County = 'Jackson'
Start_Date ='2019-01-01'
End_Date ='2020-01-01'

# set the name of final files for saving in google drive
outfilename = 'statistics_Jackson_2019'
description='Supervised-Flooded_Jackson_2019'

#Consider the crop number for checking flood impact
Corn = 1
Soybeans = 5
DurumWheat = 22
SpringWheat = 23 
WinterWheat = 24


In [4]:
#Define area of study

#state 
stateBoundary = geemap.ee.FeatureCollection('TIGER/2018/States').select('NAME')\
.filter(geemap.ee.Filter.eq('NAME', State))

#county 
CountyBoundary = geemap.ee.FeatureCollection('TIGER/2018/Counties').select('NAME')\
.filter(geemap.ee.Filter.eq('NAME', County))

#area of study
cntyGeom = CountyBoundary.geometry()

#area of study
#use this line if you want to run the code for the whole state
#and do not run next cell
rgnGeom = stateBoundary.geometry()

In [5]:
#Define intersect function for intersected county and states
def intersect(featurecollection):
  return featurecollection.intersection(cntyGeom, 1)
def addArea(feature):
  return feature.set({'areaHa': feature.geometry().area().divide(100 * 100)})

#Apply intersect function
region = stateBoundary.map(intersect)
areaAdded = region.map(addArea)
rgnGeom = areaAdded.geometry() #Final area of study

In [6]:
# collect NAIP imagery 
collection = ee.ImageCollection('USDA/NAIP/DOQQ')\
                  .filterBounds(rgnGeom) \
                  .filter(ee.Filter.date(Start_Date, End_Date))

#clip the medeain of all NAIP images to area of study
NAIP = collection.median().clip(rgnGeom)

# #Number Of Images 
# n = collection.size().getInfo();
# print(n)

# #bands
# image = collection.first()
# print('Band names: ', image.getInfo())

## Display
# Map = geemap.Map(center=(43.65, -95.17), zoom=10)
# Map.addLayer(region, {}, 'region')
# Map.addLayer(NAIP, {}, 'NAIP')
# Map

In [7]:
#Define  function for NDVI & NDWI
def addIndices(image):
    ndvi = image.normalizedDifference(['N', 'R']).rename(['NDVI'])
    ndwi = image.normalizedDifference(['G', 'N']).rename(['NDWI'])
    barren = image.normalizedDifference(['N', 'R']).lt(0.1).And(image.normalizedDifference(['G', 'N']).lt(0.2)).rename('barren')
    return image.addBands(ndvi).addBands(ndwi).addBands(barren)

#Apply Indices function
composite =addIndices(NAIP)

#bands
print('Band names: ', composite.bandNames().getInfo())

# # Display
# Map.addLayer(composite.select('NDWI'), {'min': 0.5, 'max': 0.6, 'palette': ['4E565B', '2B94D8']}, 'ndwi')
# Map.addLayer(composite.select('NDVI'), {'min': 0.2, 'max': 0.5, 'palette': ['472D67', 'F9F503']}, 'ndvi')
# Map

Band names:  ['R', 'G', 'B', 'N', 'NDVI', 'NDWI', 'barren']


In [17]:
# Cropland Supervised Classification

#USDA NASS Cropland Data Layers for validation
USDA = ee.ImageCollection('USDA/NASS/CDL') \
                  .filter(ee.Filter.date(Start_Date, End_Date)) \
                  .median()
                  

# Clip cropLandcover to area of study
Totalcropland = USDA.select('cultivated').clip(rgnGeom)
#print(type(Totalcropland))


# Make the training dataset.
points = Totalcropland.sample(**{
    'region': rgnGeom,
    'scale': 1,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

# Use these bands for prediction.
bands = [ 'R','G','B','N','NDVI', 'NDWI','barren']


# This property of the table stores the land cover labels.
label = 'cultivated'

# Overlay the points on the imagery to get training
sample = composite.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 1
})

# Adds a column of deterministic pseudorandom numbers. 
sample = sample.randomColumn()

# 70% training, 30% validation
split = 0.7

training = sample.filter(ee.Filter.lt('random', split)) #Training dataset
validation = sample.filter(ee.Filter.gte('random', split)) #Validation dataset


# Train a CART or a RandomForest classifier with default parameters.
#classifier = ee.Classifier.smileCart().train(training, label, bands)
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

# Classify the image with the same bands used for training.
cropland_classified = composite.select(bands).classify(classifier)

#Accuracy Assessment
validated = validation.classify(classifier)

Cropland_ConfusionMatrix = validated.errorMatrix('cultivated', 'classification')
#accuracy report
print('accuracy: ',Cropland_ConfusionMatrix.accuracy().getInfo())

# # Calculate consumer's accuracy, also known as user's accuracy or
# # specificity and the complement of commission error (1 − commission error).
# Wetland_ConfusionMatrix.consumersAccuracy().getInfo()

# # Display the clusters with random colors.
# Map = geemap.Map(center=(43.65, -95.17), zoom=10)
# Map.addLayer(Totalcropland.randomVisualizer(), {}, 'USDA_cultivated')
# Map.addLayer(cropland_classified.randomVisualizer(), {}, 'cropland_classified')
# Map

accuracy:  0.8568548387096774


In [18]:
# Wetland Supervised Classification

#JRC water Data Layers for training and validation
JRC_collection = ee.ImageCollection("JRC/GSW1_3/MonthlyHistory") \
                  .filter(ee.Filter.date(Start_Date, End_Date))

#Clip waterbody to area of study 
wetland = JRC_collection.median().clip(rgnGeom).select('water').eq(2)
#print(type(wetland))

# Make the training dataset.
points = wetland.sample(**{
    'region': rgnGeom,
    'scale': 1,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

# Use these bands for prediction.
bands = ['NDWI']

# This property of the table stores the land cover labels.
label = 'water'

# Overlay the points on the imagery to get training
sample = composite.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 1
})

# Adds a column of deterministic pseudorandom numbers. 
sample = sample.randomColumn()

# 70% training, 30% validation
split = 0.7

training = sample.filter(ee.Filter.lt('random', split)) #Training dataset
validation = sample.filter(ee.Filter.gte('random', split)) #Validation dataset

# Train a CART or a RandomForest classifier with default parameters.
classifier = ee.Classifier.smileCart().train(training, label, bands)
#classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

# Classify the image with the same bands used for training.
wetland_classified = composite.select(bands).classify(classifier)

#Accuracy Assessment
validated = validation.classify(classifier)

Wetland_ConfusionMatrix = validated.errorMatrix('water', 'classification')
#accuracy report
print('accuracy: ',Wetland_ConfusionMatrix.accuracy().getInfo())

# # Calculate consumer's accuracy, also known as user's accuracy or
# # specificity and the complement of commission error (1 − commission error).
# Wetland_ConfusionMatrix.consumersAccuracy().getInfo()

# # Display the clusters with random colors.
# Map = geemap.Map(center=(43.65, -95.17), zoom=10)
# Map.addLayer(wetland.randomVisualizer(), {}, 'JRC_wetland')
# Map.addLayer(wetland_classified.randomVisualizer(), {}, 'wetland_classified')
# Map

accuracy:  0.9978962131837307


In [10]:
#Unsupervised Classification

# Use these bands for clustering.
bands = [ 'NDVI', 'NDWI']

#random points
region = ee.Geometry.Point([ -95.1701,43.6652]).buffer(20000)

# Make the training dataset.
points = composite.select(bands).sample(**{
    'region': region,
    'numPixels': 5000, #number of sample points
    'seed': 0,
    'scale': 1,
    'geometries': True  
})

# Instantiate the clusterer and train it.
n_clusters = 2
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(points)

# Cluster the input using the trained clusterer.
unsupervised_classification = composite.cluster(clusterer)

# Display the clusters with random colors.
# Map.addLayer(unsupervised_classification.randomVisualizer(), {}, 'clusters')
# Map


In [11]:
# Function to Normalize Image
# Pixel Values should be between 0 and 1
# Formula is (x - xmin) / (xmax - xmin)
def normalize(image):
    bandNames = image.bandNames()
  # Compute min and max of the image
    minDict = image.reduceRegion(
    reducer= ee.Reducer.min(),
    geometry= rgnGeom,
    scale= 1,
    maxPixels= 1e9,
    bestEffort= True,
    )
    maxDict = image.reduceRegion(
    reducer= ee.Reducer.max(),
    geometry= rgnGeom,
    scale= 1,
    maxPixels= 1e9,
    bestEffort= True,
    )
    mins = ee.Image.constant(minDict.values(bandNames))
    maxs = ee.Image.constant(maxDict.values(bandNames))

    normalized = image.subtract(mins).divide(maxs.subtract(mins))
    return normalized
#Apply Normalize function
cropland_normalizedImage = normalize(cropland_classified )
wetland_normalizedImage = normalize(wetland_classified )

# # Display
# Map = geemap.Map(center=(43.65, -95.17), zoom=10)
# Map.addLayer(cropland_normalizedImage.randomVisualizer(), { }, 'cropland_normalizedclassfied')
# Map.addLayer(wetland_normalizedImage.randomVisualizer(),  { }, 'wetland_normalizedclassfied')
# Map

In [12]:
# Flooded Field Extraction
#Consider the crop number for extraction by croptype
Corn = 1
Soybeans = 5
DurumWheat = 22
SpringWheat = 23 
WinterWheat = 24

#USDA NASS Cropland Data Layers for extraction by croptype
USDA = ee.ImageCollection('USDA/NASS/CDL') \
                  .filter(ee.Filter.date(Start_Date, End_Date)) \
                  .median()

# total cropland & Flooded Fields 
Totalcropland = USDA.select('cultivated').clip(rgnGeom)

Totalcropland = Totalcropland.eq(2)
cultivated = Totalcropland.mask(Totalcropland.eq(1))
cropland = cropland_normalizedImage.mask(cultivated)
Flooded = wetland_normalizedImage.mask(cultivated)
# Flooded Fields
FloodedField_Total = Flooded.mask(Flooded.eq(1))   


# Flooded Fields by crop type
cropLands = USDA.clip(rgnGeom).select('cropland')

# Corn
mask_Corn = ee.Image(cropLands).eq(Corn)
cultivated_Corn = mask_Corn.mask(mask_Corn.eq(1))
cropland_Corn = cropland_normalizedImage.mask(cultivated_Corn)
Flooded_Corn = wetland_normalizedImage.mask(cultivated_Corn)
# Corn Flooded Fields
FloodedField_Corn = Flooded_Corn.mask(Flooded_Corn.eq(1)) 

# Soybeans 
mask_Soybeans = ee.Image(cropLands).eq(Soybeans)
cultivated_Soybeans = mask_Soybeans.mask(mask_Soybeans.eq(1))
cropland_Soybeans = cropland_normalizedImage.mask(cultivated_Soybeans)
Flooded_Soybeans = wetland_normalizedImage.mask(cultivated_Soybeans)
# Soybeans Flooded Fields
FloodedField_Soybeans = Flooded_Soybeans.mask(Flooded_Soybeans.eq(1)) 

# Wheat
mask_Wheat = ee.Image(cropLands).eq(DurumWheat).Or(cropLands.eq(SpringWheat)).Or(cropLands.eq(WinterWheat))
cultivated_Wheat = mask_Wheat.mask(mask_Wheat.eq(1))
cropland_Wheat = cropland_normalizedImage.mask(cultivated_Wheat)
Flooded_Wheat = wetland_normalizedImage.mask(cultivated_Wheat)
# Soybeans Flooded Fields
FloodedField_Wheat = Flooded_Wheat.mask(Flooded_Wheat.eq(1)) 

# USDA cultivated statistics
cultivated_total_stats = geemap.image_stats(cultivated, scale=1)
cultivated_Corn_stats = geemap.image_stats(cultivated_Corn, scale=1)
cultivated_Soybeans_stats = geemap.image_stats(cultivated_Soybeans, scale=1)
cultivated_Wheat_stats = geemap.image_stats(cultivated_Wheat, scale=1)

# Cropland statistics based on classification
cropland_total_stats = geemap.image_stats(cropland, scale=1)
cropland_Corn_stats = geemap.image_stats(cropland_Corn, scale=1)
cropland_Soybeans_stats = geemap.image_stats(cropland_Soybeans, scale=1)
cropland_Wheat_stats = geemap.image_stats(cropland_Wheat, scale=1)

# FloodedField statistics
Flooded_total_stats = geemap.image_stats(FloodedField_Total, scale=1)
Flooded_Corn_stats = geemap.image_stats(FloodedField_Corn, scale=1)
Flooded_Soybeans_stats = geemap.image_stats(FloodedField_Soybeans, scale=1)
Flooded_Wheat_stats = geemap.image_stats(FloodedField_Wheat, scale=1)

# #Display
# Map = geemap.Map(center=(43.65, -95.17), zoom=10)
# Map.addLayer(Flooded.randomVisualizer(), {}, 'Flooded')
# Map.addLayer(FloodedField_Total.randomVisualizer(), {}, 'FloodedField_Total')
# Map

In [13]:
#**************************************************************************
# Exporting Results
#**************************************************************************

# Create a Feature with {} geometry and the value we want to export.
# Use .array() to convert Confusion Matrix to an Array so it can be
# exported in a CSV file
# Create a Feature with {} geometry and the sum of Flooded pixels value we want to export. 
feature = ee.FeatureCollection([
  ee.Feature(None, {
    'cultivated_total_stats': cultivated_total_stats,
    'Flooded_total_stats': Flooded_total_stats,
    'cultivated_Corn_stats': cultivated_Corn_stats,
    'Flooded_Corn_stats': Flooded_Corn_stats,
    'cultivated_Soybeans_stats': cultivated_Soybeans_stats,
    'Flooded_Soybeans_stats': Flooded_Soybeans_stats,
    'cultivated_Wheat_stats': cultivated_Wheat_stats,
    'Flooded_Wheat_stats': Flooded_Wheat_stats,
    'cropland_total_stats' : cropland_total_stats ,
    'cropland_Corn_stats' : cropland_Corn_stats,
    'cropland_Soybeans_stats' : cropland_Soybeans_stats,
    'cropland_Wheat_stats' : cropland_Wheat_stats,
    'Cropland_classsifcation_accuracy': Cropland_ConfusionMatrix.accuracy(),
    'Cropland_classsifcation_matrix': Cropland_ConfusionMatrix.array(),
    'Wetland_classsifcation_accuracy': Wetland_ConfusionMatrix.accuracy(),
    'Wetland_classsifcation_matrix': Wetland_ConfusionMatrix.array()
  })
  ])

# Exporting Results to Drive
task_config = {
    'folder': 'GEE', # output Google Drive folder
    'fileFormat': 'csv',  
    }

#outfilename = 'statistics_Jackson_2019'

task = ee.batch.Export.table.toDrive(feature, outfilename, **task_config)
task.start()

In [14]:
#Export Geotiff to Drive
geemap.ee_export_image_to_drive(
    ee_object=Flooded.clip(rgnGeom),
#    description='Supervised-Flooded_Jackson_2019',
    description=description,
    folder='GEE',
    region=None,
    scale=1,
    max_pixels=1.0e13,
    file_format="GEO_TIFF",
    format_options={},
)

Exporting Supervised-Flooded_Jackson_2019 ...


In [15]:
## Download to local Desktop only if the size of study area is small
# out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
# out_file = os.path.join(out_dir, 'cluster22.tif')
# geemap.ee_export_image(Flooded.clip(rgnGeom), filename=out_file, scale=30)

In [16]:
##Translate Javascript to Python
# js='''
# var options = {
# lineWidth: 1,
# pointSize: 2,
# hAxis: {title: 'Classes'},
# vAxis: {title: 'Num of pixels'},
# title: 'Number of pixels in each class.'
# }; 

# var chart = ui.Chart.image.byClass({

# image : classified ,
# classBand : 'classification', 
# region : aletsch           //<-- A previousy defined line type geometry
# }).setOptions(options) ; 
# '''
# geemap.js_snippet_to_py(js, add_new_cell=True)